In [1]:
whoami; date; hostname; pwd;

ytanigaw
Fri Apr 26 14:58:49 PDT 2019
sh-102-07.int
/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/ukbb-tools/04_gwas/ref_alt


In [16]:
ml load bedtools

#### reference sequence file

In [47]:
cat /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa | head -n2

>chr1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
cat: write error: Broken pipe


In [48]:
cat /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa \
| egrep '^>' | grep -v '_'

>chr1
>chr2
>chr3
>chr4
>chr5
>chr6
>chr7
>chrX
>chr8
>chr9
>chr10
>chr11
>chr12
>chr13
>chr14
>chr15
>chr16
>chr17
>chr18
>chr20
>chrY
>chr19
>chr22
>chr21
>chrM


### Compare UKBB's A1/A2 with the reference sequence

#### tl;dr: UKBB A1 column (5-th column in the bim file) matches with the reference sequence

We will take chr22 as an example case first. This is the bim file from the UKBB.

In [27]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr22_v2.bim | head -n10

22	rs62224618	0	16057417	C	T
22	Affx-52233492	0	16287538	G	C
22	rs200923174	0	16287557	C	G
22	Affx-80289661	0	16287585	AC	A
22	Affx-80289662	0	16287663	TC	T
22	Affx-52336937	0	16287779	CT	C
22	rs138257042	0	16449075	G	A
22	rs116911124	0	16495833	C	A
22	rs117578132	0	16595552	C	A
22	rs117982183	0	16648658	C	T
cat: write error: Broken pipe


We query the reference fasta file with `bedtools getfasta` command, which requires UCSC bed file as input so I converted the PLINK bim file to UCSC bed file using awk. 
Specifically, PLINK bim is on 1-based coordinate and UCSC bed is on 0-based semi-intervals. 


In the UCSC bed file, we generated the name in the following format:
`<variant ID>:<chromosome>:<position (1-based index)>:<A1 (5-th column in bim file)>:<A2 (6-th column in bim file)>`

We pipe this UCSC bed file into `bedtools getfasta` command to get the reference sequence, which is shown at the last column of the input


In [45]:
cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr22_v2.bim | head -n10 \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($5), $2 sep chr $1 sep $4 sep $5 sep $6}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut

cat: write error: Broken pipe
chr22	16057416	16057417	rs62224618:chr22:16057417:C:T	C
chr22	16287537	16287538	Affx-52233492:chr22:16287538:G:C	G
chr22	16287556	16287557	rs200923174:chr22:16287557:C:G	C
chr22	16287584	16287586	Affx-80289661:chr22:16287585:AC:A	AC
chr22	16287662	16287664	Affx-80289662:chr22:16287663:TC:T	TC
chr22	16287778	16287780	Affx-52336937:chr22:16287779:CT:C	CT
chr22	16449074	16449075	rs138257042:chr22:16449075:G:A	G
chr22	16495832	16495833	rs116911124:chr22:16495833:C:A	c
chr22	16595551	16595552	rs117578132:chr22:16595552:C:A	c
chr22	16648657	16648658	rs117982183:chr22:16648658:C:T	c


As you can see above, the reference sequence matches with the A1 column in the bim file.

We now test all autosomes to see if there are mismatches.

In [44]:
for c in $(seq 1 22) ; do cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr${c}_v2.bim ; done \
| awk -v OFS='\t' -v chr="chr" -v sep=":" '{print chr $1, $4-1, $4-1+length($5), $2 sep chr $1 sep $4 sep $5 sep $6}' \
| bedtools getfasta -bedOut -fi /oak/stanford/groups/mrivas/public_data/genomes/hg19/hg19.fa -bed /dev/stdin -bedOut \
| tr ":" "\t" | awk '$7 != toupper($9)'

chr4	7435304	7435305	Affx-24919403	chr4	7435305	T	C	A
chr7	150896002	150896003	Affx-29977087	chr7	150896003	T	G	C
chr12	53011954	53011955	Affx-8165091	chr12	53011955	T	C	G
chr21	46354974	46354975	Affx-18140622	chr21	46354975	A	G	C
chr22	39218763	39218764	Affx-19589717	chr22	39218764	T	C	G


In [49]:
for c in $(seq 1 22) ; do cat $OAK/private_data/ukbb/24983/cal/pgen/ukb24983_cal_chr${c}_v2.bim ; done | wc

 784256 4705536 22514697


There are 5/784256 exceptions, which can be due to minor version difference of the reference genome (we used GRCh37.p13, but UKBB may have used other minor versions, such as GRCh37.p12)